In [83]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder,MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
import datetime

In [2]:
from google.colab import files
upload = files.upload()

customer_csv = pd.read_csv('Customers.csv')
product_csv = pd.read_csv('Products.csv')
transactions_csv = pd.read_csv('Transactions.csv')

Saving Customers.csv to Customers.csv
Saving Data Science Intern _ Assignment (1).pdf to Data Science Intern _ Assignment (1).pdf
Saving Products.csv to Products.csv
Saving Transactions.csv to Transactions.csv


**LOOKALIKE MODEL**

In [20]:
transactions_csv['TransactionDate'] = pd.to_datetime(transactions_csv['TransactionDate'])
model_data_frame = transactions_csv.groupby('CustomerID')['TransactionDate'].max().reset_index()
date = pd.to_datetime('2025-01-01')
model_data_frame['RecentTransactionDays'] = (date - model_data_frame['TransactionDate']).dt.days
print(model_data_frame)

    CustomerID     TransactionDate  RecentTransactionDays
0        C0001 2024-11-02 17:04:16                     59
1        C0002 2024-12-03 01:41:41                     28
2        C0003 2024-08-24 18:54:04                    129
3        C0004 2024-12-23 14:13:52                      8
4        C0005 2024-11-04 00:30:22                     57
..         ...                 ...                    ...
194      C0196 2024-12-15 03:43:35                     16
195      C0197 2024-12-27 18:20:31                      4
196      C0198 2024-10-04 18:31:12                     88
197      C0199 2024-10-26 00:01:58                     66
198      C0200 2024-12-11 03:05:50                     20

[199 rows x 3 columns]


In [29]:
model_data_frame = model_data_frame.merge(customer_csv[['CustomerID','SignupDate']],on="CustomerID",how='inner')

MergeError: Passing 'suffixes' which cause duplicate columns {'SignupDate_x'} is not allowed.

In [32]:
model_data_frame['history'] = (date - pd.to_datetime(model_data_frame['SignupDate'])).dt.days
#model_data_frame = model_data_frame.drop(columns=['SignupDate_x'])
#model_data_frame = model_data_frame.drop(columns=['SignupDate_y'])
print(model_data_frame)

    CustomerID     TransactionDate  RecentTransactionDays  SignupDate  history
0        C0001 2024-11-02 17:04:16                     59  2022-07-10      906
1        C0002 2024-12-03 01:41:41                     28  2022-02-13     1053
2        C0003 2024-08-24 18:54:04                    129  2024-03-07      300
3        C0004 2024-12-23 14:13:52                      8  2022-10-09      815
4        C0005 2024-11-04 00:30:22                     57  2022-08-15      870
..         ...                 ...                    ...         ...      ...
194      C0196 2024-12-15 03:43:35                     16  2022-06-07      939
195      C0197 2024-12-27 18:20:31                      4  2023-03-21      652
196      C0198 2024-10-04 18:31:12                     88  2022-02-27     1039
197      C0199 2024-10-26 00:01:58                     66  2022-12-03      760
198      C0200 2024-12-11 03:05:50                     20  2023-06-11      570

[199 rows x 5 columns]


In [33]:
regions = {
    'South America' : 0,
    'North America' : 1,
    'Europe' : 2,
    'Asia' : 3
}

model_data_frame['Region'] = customer_csv['Region'].map(regions)
print(model_data_frame)

    CustomerID     TransactionDate  RecentTransactionDays  SignupDate  \
0        C0001 2024-11-02 17:04:16                     59  2022-07-10   
1        C0002 2024-12-03 01:41:41                     28  2022-02-13   
2        C0003 2024-08-24 18:54:04                    129  2024-03-07   
3        C0004 2024-12-23 14:13:52                      8  2022-10-09   
4        C0005 2024-11-04 00:30:22                     57  2022-08-15   
..         ...                 ...                    ...         ...   
194      C0196 2024-12-15 03:43:35                     16  2022-06-07   
195      C0197 2024-12-27 18:20:31                      4  2023-03-21   
196      C0198 2024-10-04 18:31:12                     88  2022-02-27   
197      C0199 2024-10-26 00:01:58                     66  2022-12-03   
198      C0200 2024-12-11 03:05:50                     20  2023-06-11   

     history  Region  
0        906       0  
1       1053       3  
2        300       0  
3        815       0  
4       

In [40]:
total_spent_data_frame = transactions_csv.groupby('CustomerID')['TotalValue'].sum().reset_index()
merged_data = model_data_frame.merge(total_spent_data_frame, on='CustomerID', how='inner')
merged_data.rename(columns={'TotalValue': 'total_spent'}, inplace=True)
model_data_frame = merged_data
print(model_data_frame)

    CustomerID     TransactionDate  RecentTransactionDays  SignupDate  \
0        C0001 2024-11-02 17:04:16                     59  2022-07-10   
1        C0002 2024-12-03 01:41:41                     28  2022-02-13   
2        C0003 2024-08-24 18:54:04                    129  2024-03-07   
3        C0004 2024-12-23 14:13:52                      8  2022-10-09   
4        C0005 2024-11-04 00:30:22                     57  2022-08-15   
..         ...                 ...                    ...         ...   
194      C0196 2024-12-15 03:43:35                     16  2022-06-07   
195      C0197 2024-12-27 18:20:31                      4  2023-03-21   
196      C0198 2024-10-04 18:31:12                     88  2022-02-27   
197      C0199 2024-10-26 00:01:58                     66  2022-12-03   
198      C0200 2024-12-11 03:05:50                     20  2023-06-11   

     history  Region  total_spent  
0        906       0      3354.52  
1       1053       3      1862.74  
2        300   

In [41]:
total_spent_data_frame = transactions_csv.groupby('CustomerID')['Quantity'].sum().reset_index()
merged_data = model_data_frame.merge(total_spent_data_frame, on='CustomerID', how='inner')
merged_data.rename(columns={'TotalQuantity': 'Total_Qty'}, inplace=True)
model_data_frame = merged_data
print(model_data_frame)

    CustomerID     TransactionDate  RecentTransactionDays  SignupDate  \
0        C0001 2024-11-02 17:04:16                     59  2022-07-10   
1        C0002 2024-12-03 01:41:41                     28  2022-02-13   
2        C0003 2024-08-24 18:54:04                    129  2024-03-07   
3        C0004 2024-12-23 14:13:52                      8  2022-10-09   
4        C0005 2024-11-04 00:30:22                     57  2022-08-15   
..         ...                 ...                    ...         ...   
194      C0196 2024-12-15 03:43:35                     16  2022-06-07   
195      C0197 2024-12-27 18:20:31                      4  2023-03-21   
196      C0198 2024-10-04 18:31:12                     88  2022-02-27   
197      C0199 2024-10-26 00:01:58                     66  2022-12-03   
198      C0200 2024-12-11 03:05:50                     20  2023-06-11   

     history  Region  total_spent  Quantity  
0        906       0      3354.52        12  
1       1053       3      1862.

In [42]:
total_spent_data_frame = transactions_csv.groupby('CustomerID')['Quantity'].sum().reset_index()
merged_data = model_data_frame.merge(total_spent_data_frame, on='CustomerID', how='inner')
merged_data.rename(columns={'TotalQuantity': 'Total_Qty'}, inplace=True)
model_data_frame = merged_data
print(model_data_frame)

In [45]:
uniques_products = product_csv['Category'].unique()
print(uniques_products)
uniques_products = {
    'Books' : 0,
    'Electronics' : 1,
    'Home Decor' : 2,
    'Clothing' : 3
}


['Books' 'Electronics' 'Home Decor' 'Clothing']
0     0
1     1
2     0
3     2
4     3
     ..
95    1
96    0
97    1
98    0
99    3
Name: Category, Length: 100, dtype: int64


In [52]:

customer_to_products = transactions_csv.groupby(['CustomerID','ProductID'])['Quantity'].sum().reset_index()
#print(customer_to_products)
max_quantity_for_each_customer = customer_to_products.groupby('CustomerID')['Quantity'].transform('max')
most_bought_products = customer_to_products[customer_to_products['Quantity'] == max_quantity_for_each_customer]
most_bought_products_per_customer = most_bought_products.groupby('CustomerID')['ProductID'].apply(list).reset_index()
merge_product_and_most_brought = most_bought_products.merge(product_csv, on='ProductID', how='left')
final_data = merge_product_and_most_brought.groupby('CustomerID')['Category'].apply(list).reset_index()
final_data['Category'] = final_data['Category'].apply(lambda x: [uniques_products[cat] for cat in x])
print(final_data)

    CustomerID   Category
0        C0001     [2, 1]
1        C0002        [2]
2        C0003     [1, 3]
3        C0004  [0, 2, 1]
4        C0005        [2]
..         ...        ...
194      C0196        [2]
195      C0197        [1]
196      C0198        [3]
197      C0199        [2]
198      C0200  [3, 0, 2]

[199 rows x 2 columns]


In [57]:
model_data_frame = model_data_frame.merge(final_data,on='CustomerID',how='inner')


MergeError: Passing 'suffixes' which cause duplicate columns {'Category_x'} is not allowed.

In [60]:
model_data_frame = model_data_frame.drop(columns=['Category_x'])
model_data_frame = model_data_frame.drop(columns=['Category_y'])

Index(['CustomerID', 'TransactionDate', 'RecentTransactionDays', 'SignupDate',
       'history', 'Region', 'total_spent', 'Quantity', 'Category_x',
       'Category_y', 'Category'],
      dtype='object')


In [62]:
print(model_data_frame)

    CustomerID     TransactionDate  RecentTransactionDays  SignupDate  \
0        C0001 2024-11-02 17:04:16                     59  2022-07-10   
1        C0002 2024-12-03 01:41:41                     28  2022-02-13   
2        C0003 2024-08-24 18:54:04                    129  2024-03-07   
3        C0004 2024-12-23 14:13:52                      8  2022-10-09   
4        C0005 2024-11-04 00:30:22                     57  2022-08-15   
..         ...                 ...                    ...         ...   
194      C0196 2024-12-15 03:43:35                     16  2022-06-07   
195      C0197 2024-12-27 18:20:31                      4  2023-03-21   
196      C0198 2024-10-04 18:31:12                     88  2022-02-27   
197      C0199 2024-10-26 00:01:58                     66  2022-12-03   
198      C0200 2024-12-11 03:05:50                     20  2023-06-11   

     history  Region  total_spent  Quantity   Category  
0        906       0      3354.52        12     [2, 1]  
1       1

Start to Normalise

In [77]:
scaler = MinMaxScaler()
model_data_frame[['RecentTransactionDays', 'history', 'total_spent', 'Quantity']] = scaler.fit_transform(
    model_data_frame[['RecentTransactionDays', 'history', 'total_spent', 'Quantity']]
)

In [75]:
categories = ['Books', 'Electronics', 'Home Decor', 'Clothing']

mlb_class = MultiLabelBinarizer(classes=range(len(categories)))
category_encoded_data = pd.DataFrame(mlb_class.fit_transform(model_data_frame['Category']), columns=categories)
model_data_frame = pd.concat([model_data_frame.drop(columns=['Category']), category_encoded_data], axis=1)

KeyError: 'Category'

In [89]:
one_hot_encoder = OneHotEncoder()
region_coded =  pd.get_dummies(model_data_frame['Region'], prefix='Region').astype(int)
model_data_frame = pd.concat([model_data_frame, region_coded], axis=1)


In [90]:
print(model_data_frame)

    CustomerID     TransactionDate  RecentTransactionDays  SignupDate  \
0        C0001 2024-11-02 17:04:16               0.155125  2022-07-10   
1        C0002 2024-12-03 01:41:41               0.069252  2022-02-13   
2        C0003 2024-08-24 18:54:04               0.349030  2024-03-07   
3        C0004 2024-12-23 14:13:52               0.013850  2022-10-09   
4        C0005 2024-11-04 00:30:22               0.149584  2022-08-15   
..         ...                 ...                    ...         ...   
194      C0196 2024-12-15 03:43:35               0.036011  2022-06-07   
195      C0197 2024-12-27 18:20:31               0.002770  2023-03-21   
196      C0198 2024-10-04 18:31:12               0.235457  2022-02-27   
197      C0199 2024-10-26 00:01:58               0.174515  2022-12-03   
198      C0200 2024-12-11 03:05:50               0.047091  2023-06-11   

      history  Region  total_spent  Quantity  Books  Electronics  Home Decor  \
0    0.842204       0     0.308942  0.35483

COSINE SIMILARITY

In [92]:
cosine_data_frame = model_data_frame[['RecentTransactionDays','history','Region_0','Region_1','Region_2','Region_3','total_spent','Quantity'
,'Books','Electronics','Home Decor','Clothing']]

In [98]:
features = cosine_data_frame.values
cosine_similarity_values = cosine_similarity(features)
similarity_computed = pd.DataFrame(cosine_similarity_values, index=model_data_frame["CustomerID"], columns=model_data_frame["CustomerID"])

In [99]:
print(similarity_computed)

CustomerID     C0001     C0002     C0003     C0004     C0005     C0006  \
CustomerID                                                               
C0001       1.000000  0.570668  0.681637  0.881563  0.555099  0.502719   
C0002       0.570668  1.000000  0.141030  0.502107  0.993814  0.186132   
C0003       0.681637  0.141030  1.000000  0.616207  0.131101  0.834720   
C0004       0.881563  0.502107  0.616207  1.000000  0.481941  0.471923   
C0005       0.555099  0.993814  0.131101  0.481941  1.000000  0.171266   
...              ...       ...       ...       ...       ...       ...   
C0196       0.859098  0.659660  0.464873  0.773323  0.638757  0.583247   
C0197       0.530411  0.253205  0.453688  0.477170  0.219769  0.147441   
C0198       0.260739  0.327529  0.434897  0.205350  0.293938  0.528050   
C0199       0.548626  0.636082  0.136088  0.483693  0.625207  0.172134   
C0200       0.407885  0.455897  0.366758  0.591809  0.447347  0.451438   

CustomerID     C0007     C0008     C0

In [101]:
lookalikes_map = {}
for customer_id in similarity_computed.index[:20]:
    top_3 = similarity_computed.loc[customer_id].drop(customer_id).nlargest(3)
    lookalikes_map[customer_id] = list(top_3.items())

lookalike_csv_datas = pd.DataFrame({
    "CustomerID":lookalikes_map.keys(),
    "top_lookalikes":lookalikes_map.values()
})

In [103]:
print(lookalike_csv_datas)

   CustomerID                                     top_lookalikes
0       C0001  [(C0147, 0.996340184702442), (C0085, 0.9215160...
1       C0002  [(C0159, 0.9960733974253128), (C0005, 0.993813...
2       C0003  [(C0096, 0.8523912755648813), (C0104, 0.840849...
3       C0004  [(C0169, 0.90774491241042), (C0099, 0.89794839...
4       C0005  [(C0159, 0.9976759081958886), (C0002, 0.993813...
5       C0006  [(C0129, 0.990495805175904), (C0031, 0.9855700...
6       C0007  [(C0005, 0.9928333774281534), (C0002, 0.983763...
7       C0008  [(C0109, 0.9473911085631465), (C0042, 0.936543...
8       C0009  [(C0170, 0.9583651313514724), (C0111, 0.939371...
9       C0010  [(C0135, 0.9920329863369449), (C0062, 0.973692...
10      C0011  [(C0158, 0.9791019128454793), (C0006, 0.966036...
11      C0012  [(C0163, 0.9654795389216109), (C0052, 0.944894...
12      C0013  [(C0189, 0.975638365387258), (C0082, 0.8611584...
13      C0014  [(C0127, 0.8861876352735284), (C0121, 0.886109...
14      C0015  [(C0116, 0

In [104]:
lookalike_csv_datas.to_csv('Lookalike.csv',index=False)